In [1]:
from foundations.projectables import SympyProjectable
from foundations.projectables import ProjectableModel
from modeling.arghandling import EncodedFunction, Encoder
from modeling.compute import create_vars
import numpy as np
from pint import UnitRegistry

In [2]:
ureg = UnitRegistry()
z_table = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 25, 30, 40])*1e3
T_table_celsius = np.array([15, 8.5,2,-4.49,-10.98,-17.47,-23.96,-30.45,-36.94,-43.42,-49.9,-56.5,-56.5,-51.6,-46.64,-22.8])
T_table = ureg.Quantity(T_table_celsius, 'degC').to('K').magnitude
G_table = np.array([9.807,9.804,9.801,9.797,9.794,9.791,9.788,9.785,9.782,9.779,9.776,9.761,9.745,9.73,9.715,9.684])
P_table = np.array([10.13,8.988,7.95,7.012,6.166,5.405,4.722,4.111,3.565,3.08,2.65,1.211,0.5529,0.2549,0.1197,0.0287])*1e4
rho_table = np.array([1.225,1.112,1.007,0.9093,0.8194,0.7364,0.6601,0.59,0.5258,0.4671,0.4135,0.1948,0.08891,0.04008,0.01841,0.003996])

Tinterp = lambda x: (np.interp(x, z_table, T_table),)
Ginterp = lambda x: (np.interp(x, z_table, G_table),)
Pinterp = lambda x: (np.interp(x, z_table, P_table),)
rhointerp = lambda x: (np.interp(x, z_table, rho_table),)

In [3]:
k, R, RHe, p, C_D, G0 = (1.38e-23, 8.314, 2077.1, 8/5, 0.47, 9.81)
mm_He, mm_H2, rho_He, rho_H2 = (6.65e-27, 3.35e-27, 0.1786, 0.0899)
rho0 = rhointerp(0)
alpha, vr, m_vhc, m_parafoil = (1, 6, 4545, 500)
z, m_b, Vc = create_vars('z m_b V_c')

In [4]:
Balloon = ProjectableModel()
pz = Balloon.VarRaw('p_z', EncodedFunction(Pinterp, Encoder((z,))))
Tz = Balloon.VarRaw('T_z', EncodedFunction(Tinterp, Encoder((z,))))

rhoz = Balloon.Var(r'\rhoz', pz/(R*Tz))

# density of lifting gas at max altitude z
rho_LGz = Balloon.Var(r'\rho_{LGz}', pz*(alpha*mm_He+(1-alpha)*mm_H2)/(k*Tz))
m_t = Balloon.Var('m_t', m_vhc+m_parafoil+m_b)
Vz = Balloon.Var('V_z', m_t/(rhoz-rho_LGz)) # how much can we fit?
rz = Balloon.Var('r_z', (Vz/(2*np.pi))**(1/3)) # how big is the balloon?
hz =  Balloon.Var('h_z', 2*(3/2)*rz) # how tall is the balloon?
m_lg = Balloon.Var('m_{lg}', Vz*rho_LGz) # how much lifting gas do we need?

rho_LG0 = Balloon.Var(r'\rho_{LG0}', alpha*rho_He+(1-alpha)*rho_H2) # technically a parameter
VLG = Balloon.Var('V_{LG}', m_lg/rho_LG0) #volume of lifting gas should be at sea level

# V0 = Balloon.Var('V_0', m_t/rho0)  # based on buoyancy; should be m_t/(rho0-rho_LG0)
# #r0 = Balloon.Var('r_0', (3*V0/(4*np.pi))**(1/3))

# rc = Balloon.Var('r_c', (3*Vc/(4*np.pi))**(1/3))
# ce = Balloon.Var('c_e', 3*C_D*vr**2/(8*rc*G0)) 
# Balloon.add_equation(Vc, m_t/(rho0*(1-ce)-rho_LG0)) # volume of LG required for raise speed vr



# m_lg0 = Balloon.Var('m_{lg0}', Vc*rho_LG0)
# S = Balloon.Var('S', 4*np.pi*(((rz**2)**p
#                                +2*abs(rz)**p*(abs(hz)/2)**p)/3)**(1/p))

# m_l = Balloon.VarRaw('m_l', EncodedFunction(
#     lambda ml1, ml2: max(ml1,ml2), Encoder(m_lg,m_lg0)))
# Vc2 = Balloon.Var('V_{c2}', 4/3*np.pi*rc**3) 

In [5]:
Balloon.functional().dict_in_dict_out({z: 10e3, m_b:100})

{p_z: 26500.0,
 T_z: 223.24999999999997,
 \rhoz: 14.277244147070741,
 \rho_{LGz}: 0.05720012334258398,
 m_t: 5145,
 V_z: 361.81322585322795,
 r_z: 3.8616080392081633,
 h_z: 11.584824117624489,
 m_{lg}: 20.695761145782832,
 \rho_{LG0}: 0.1786,
 V_{LG}: 115.87772198086691}

In [ ]:
model = Model()
m = model.root
#m = addsolver(r)
m_b = Var('m_b', unit='kg')
m_t = Var('m_t', unit='kg')

rho_He = Par('\\rho_{He}', 0.1786, 'kg/m^3')
rho_H2 = Par('\\rho_{H2}', 0.08988, 'kg/m^3')
rho_LG0 = adda(m, '\\rho_{LG0}', alpha*rho_He+(1-alpha)*rho_H2)
VLG = adda(m, 'V_{LG}', m_lg/rho_LG0)
RHe = Par('R_{He}', 2077.1, 'J/kg/K')
T0 = Par('T_0', Tinterp(0), 'K')
P0 = Par('P_0', Pinterp(0), 'Pa')
rho0 = Par('\\rho_0', ρinterp(0), 'kg/m^3')
V0 = adda(m, 'V_0', m_t/(rho0-rho_LG0)) #fixed from P0/(RHe*T0) which is used in the model
r0 = adda(m, 'r_0', (3*V0/(4*np.pi))**(1/3), unit='m', forceunit=True)
rc = Var('r_c', unit='m')
C_D = Par('C_D', 0.47)
vr = Par('v', 6, 'm/s')
G0 = Par('G_0', Ginterp(0), 'm/s^2')
ce = adda(m, 'c_e', 3*C_D*vr**2/(8*rc*G0)) 
Vc = adda(m, 'V_c', m_t/(rho0*(1-ce)-rho_LG0)) # this is where the loop gets introduced with respect to volume
m_lg0 = adda(m, 'm_{lg0}', Vc*rho_LG0)
m_l = adda(m, 'm_l', lambda ml1, ml2: max(ml1,ml2), (m_lg,m_lg0))
Vc2 = adda(m, 'V_{c2}', 4/3*np.pi*rc**3) 
addf(m, Vc2-Vc)
setsolvefor(m, [rc])
t_LLDPE = Par('tL', 25.4e-6*1, 'm')
rho_LLDPE = Par('rhoL', 925, 'kg/m^3')
addf(m, m_b-2*(3/2)*S*t_LLDPE*rho_LLDPE)
setsolvefor(m, [m_b])
m_lgc = adda(m, 'm_lgc', m_lg*(Vc/VLG+m_b/m_t))
adda(m, 'V_{LGc}', m_lg0/rho_LG0);